#### Configuración de token SAS para la conexión con Azure

In [ ]:
spark.conf.set("fs.azure.account.auth.type.tweetsfiles.dfs.core.windows.net", "SAS")
spark.conf.set("fs.azure.sas.token.provider.type.tweetsfiles.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set("fs.azure.sas.fixed.token.tweetsfiles.dfs.core.windows.net", "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-07-07T22:27:21Z&st=2024-05-04T14:27:21Z&spr=https&sig=hNw")

#### Instalación de dependencias necesarias
* Libreria de sentimiento utilizada: https://github.com/sentiment-analysis-spanish/sentiment-spanish/blob/master/README.md

In [ ]:
#%pip install scikit-learn
#%pip install scipy
#%pip install sentiment-analysis-spanish
# Reset de python
#dbutils.library.restartPython()

#### Imports

In [ ]:
from sentiment_analysis_spanish import sentiment_analysis
from pyspark import SparkContext
import pyspark.sql.functions as fn
import pyspark.sql.types as tipos
sc = SparkContext.getOrCreate()

In [ ]:
filename = dbutils.widgets.get('file_name_origin')
#routename = dbutils.widgets.get('save_route_databricks')

#### Selección de datos para sacar la columna del sentimiento y la URL de cada tweet

In [ ]:

esquema = tipos.StructType([
    tipos.StructField("user_id",tipos.LongType(),True),
    tipos.StructField("username",tipos.StringType(),True),
    tipos.StructField("tweet_id",tipos.LongType(), True),
    tipos.StructField("created_at",tipos.StringType(),True),
    tipos.StructField("text",tipos.StringType(),True),
    tipos.StructField("language",tipos.StringType(),True),
    tipos.StructField("reply_count",tipos.IntegerType(),True),
    tipos.StructField("retweet_count",tipos.IntegerType(),True),
    tipos.StructField("likes",tipos.IntegerType(),True),
    tipos.StructField("view_count",tipos.FloatType(),True),
    tipos.StructField("location",tipos.StringType(),True),
    tipos.StructField("hashtags",tipos.StringType(),True),
    tipos.StructField("in_reply_to",tipos.StringType(),True),
    tipos.StructField("quote",tipos.StringType(),True),
    tipos.StructField("quote_count",tipos.IntegerType(),True),
    tipos.StructField("possibly_sensetive",tipos.BooleanType(),True),
    tipos.StructField("latitud",tipos.FloatType(),True),
    tipos.StructField("longitud",tipos.FloatType(),True),
    tipos.StructField("temas",tipos.StringType(),True),
    tipos.StructField("primaryID",tipos.IntegerType(),False)
])

#Indicamos el archivo a procesar
#filename="viewnext.csv"
file_location = f'abfss://rawdata@tweetsfiles.dfs.core.windows.net/processed_dataflow_csv/{filename}'
raw_tweets = spark.read.schema(esquema).format("csv").option("delimiter", "*").option("header", "true").load(file_location).distinct()

#Creamos la columna URL para poder ver cada tweet en Twitter
tweets = (raw_tweets.select("primaryID","user_id","username", "tweet_id", "created_at", "retweet_count", "likes", "view_count", "hashtags", "text", "location", "temas", "latitud", "longitud")
          .filter(raw_tweets.language == "es")
          .withColumn("URL", fn.regexp_extract("text", r'(http)\S+', 0))
          .withColumn("text", fn.regexp_replace(fn.translate("text", "áéíóú", "aeiou"), r'(http)\S+', ""))
          )

# Creacion de instancias
puntos_sentimientos = sentiment_analysis.SentimentAnalysisSpanish()

# Definimos la UDF para sacar la puntuación de los sentimientos
def apply_sentiment(text):
    return float(puntos_sentimientos.sentiment(text))
sentiment_udf = udf(apply_sentiment, tipos.FloatType())

# Definimos la UDF para sacar modificar los valores nulos
def apply_clear_null(text):
    if text is None:
        text = "No especificado"
    return text
clear_null_udf = udf(apply_clear_null)

# Definimos la UDF para sacar modificar los valores nulos de la geolocalizacion
def apply_clear_null_geo(text):
    value = float(0)
    if text is None:
        return value
    else:
        return text
clear_null_geo_udf = udf(apply_clear_null_geo)

# Definimos la UDF para resumir en "positivo - neutral - negativo"
def result_sentiment(score):
    resultado = "positivo"
    if (score < 1.00e-5):
        resultado = "negativo"
    elif (score >= 1.00e-5 and score < 0.33):
        resultado = "neutral"
    return resultado

sentiment_udf = udf(apply_sentiment, tipos.FloatType())
sentiment_result_udf = udf(result_sentiment)

# Aplicamos todas las UDF a la columna 'text'
tweets_st = (tweets
             .withColumn("sentiment_score", sentiment_udf("text"))
             .withColumn("sentiment_result", sentiment_result_udf("sentiment_score"))
             .withColumn("location", clear_null_udf("location"))
             .withColumn("hashtags", clear_null_udf("hashtags"))
             .withColumn("latitud", clear_null_geo_udf("latitud"))
             .withColumn("longitud", clear_null_geo_udf("longitud"))
             )



#display(tweets_st)


#### Persistir los datos en el DataLake creando un único CSV y eliminando los archivos residuales

In [ ]:
#ruta = f'abfss://outputdata@tweetsfiles.dfs.core.windows.net/{routename}'
ruta = f'abfss://outputdata@tweetsfiles.dfs.core.windows.net/lib/{filename}'
ruta_temporal = f'abfss://outputdata@tweetsfiles.dfs.core.windows.net/tmp'
tweets_st.repartition(1).write.format("CSV").option("header", "true").option("delimiter", "*").mode("overwrite").save(ruta_temporal)

contenedor = dbutils.fs.ls(ruta_temporal)
for fichero in contenedor:
  if '.csv' in fichero.name:
    file = fichero.name

dbutils.fs.cp(f'{ruta_temporal}/{file}', ruta)
dbutils.fs.rm(ruta_temporal, True)